In [ ]:
import globus_automate_client
import mdf_toolbox
import json
import time
from deriva.core import DerivaServer

In [ ]:
from getpass import getpass
aws_id = getpass("AWS ID: ")
aws_secret = getpass("AWS Secret: ")

In [ ]:
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

In [ ]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [ ]:
deriva_default_acls = {
    "owner": [
        # Demo Admin
        "https://auth.globus.org/5a773142-e2ed-11e8-a017-0e8017bdda58",
        # Demo Creator
        "https://auth.globus.org/bc286232-a82c-11e9-8157-0ed6cb1f08e0"
    ],  
    "insert": [
        # Demo Curator
        "https://auth.globus.org/a5cfa412-e2ed-11e8-a768-0e368f3075e8",
        # Demo Writer
        "https://auth.globus.org/caa11064-e2ed-11e8-9d6d-0a7c1eab007a"
    ],  
    "update": [
        # Demo Curator
        "https://auth.globus.org/a5cfa412-e2ed-11e8-a768-0e368f3075e8",
        # Demo Writer
        "https://auth.globus.org/caa11064-e2ed-11e8-9d6d-0a7c1eab007a"
    ],  
    "delete": [
        # Demo Curator
        "https://auth.globus.org/a5cfa412-e2ed-11e8-a768-0e368f3075e8",
        # Demo Writer
        "https://auth.globus.org/caa11064-e2ed-11e8-9d6d-0a7c1eab007a"
    ],  
    "select": [
        # Demo Reader
        "https://auth.globus.org/b9100ea4-e2ed-11e8-8b39-0e368f3075e8",
        # ISRD Staff
        "https://auth.globus.org/176baec4-ed26-11e5-8e88-22000ab4b42b",
        # ISRD Testers
        "https://auth.globus.org/9d596ac6-22b9-11e6-b519-22000aef184d"
    ],  
    "enumerate": [
        # All (?)
        "*"
    ]
}

In [ ]:
# TODO
author_acls = deriva_default_acls
curator_acls = deriva_default_acls
cfde_acls = deriva_default_acls

In [ ]:
curator_emails = ["rick@ucsd.edu"]
#curator_emails = ["jgaff@uchicago.edu"]
curator_group = "urn:globus:groups:id:1f2f4a2d-7363-11ea-b803-0ef992ed7ca1"  # CFDE Curator Group
#cfde_emails = [""]
cfde_emails = curator_emails
cfde_group = "urn:globus:groups:id:5589052d-7363-11ea-b1e3-0a990c2810ad"  # CFDE Community Curators
sender_email = "jgaff@uchicago.edu"
admin_email = "jgaff@uchicago.edu"

In [ ]:
curate_subflow_id = "cb343d90-d772-41e1-8168-5334ffcc037f"
curate_subflow = flows_client.get_flow(curate_subflow_id)
curate_subflow_scope = curate_subflow["globus_auth_scope"]
curate_subflow_url = curate_subflow["action_url"]

deriva_ap_url = "https://ap-dev.nih-cfde.org/"
deriva_ap_scope = "https://auth.globus.org/scopes/21017803-059f-4a9b-b64c-051ab7c1d05d/demo"


In [ ]:
author_curate_text = ("'Your submission has been ingested to DERIVA here: ' + catalog_link "
                      "+ ' . Please review it. \\n If you are ready to submit this catalog, "
                      "please accept the submission. Otherwise, reject the submission and resubmit.'")
curator_curate_text = ("'A new submission has been ingested to DERIVA here: ' + catalog_link "
                       "+ ' . Please review it for quality. \\n If this catalog should be approved "
                       "and included in DERIVA, please accept the submission. Otherwise, "
                       "please reject the submission. The submission author will be notified.'")
failure_text = ("'Your CFDE submission (' + action_id + ') failed to ingest into DERIVA "
                "with this error:\\n' + error")

success_email_template = ("Your CFDE submission ($action_id) has been successfully ingested, "
                          "and can be viewed here: $catalog_link \n Thank you.")

In [ ]:
full_curation_flow_def = {
    "api_version": "1.0",
    "definition": {
        "StartAt": "CreateVars",
        "States": {
            "CreateVars": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        #TODO - how to add author's identity to the ACLs?
                        "expression": "author_acls",
                        "arguments": {
                            "author_acls": author_acls
                        },
                        "result_path": "author_acls"
                    }, {
                        "expression": "curator_acls",
                        "arguments": {
                            "curator_acls": curator_acls
                        },
                        "result_path": "curator_acls"
                    }, {
                        "expression": "cfde_acls",
                        "arguments": {
                            "cfde_acls": cfde_acls
                        },
                        "result_path": "cfde_acls"
                    }, {
                        "expression": "user_acls if user_acls else default_acls",
                        "arguments": {
                            "user_acls.$": "$.final_acls",
                            "default_acls": deriva_default_acls
                        },
                        "result_path": "final_acls"
                    }]
                },
                "ResultPath": "$.vars",
                "WaitTime": 86400,
                "Next": "ChooseTransfer"
            },
            "ChooseTransfer": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.source_endpoint_id",
                    "BooleanEquals": False,
                    "Next": "CreateNoTransferVars"
                }],
                "Default": "TransferData"
            },
            "TransferData": {
                "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
                "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "source_endpoint_id.$": "$.source_endpoint_id",
                    "destination_endpoint_id.$": "$.cfde_ep_id",
                    "transfer_items": [{
                        "source_path.$": "$.source_path",
                        "destination_path.$": "$.cfde_ep_path",
                        "recursive.$": "$.is_directory"
                    }]
                },
                "ResultPath": "$.TransferResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CreateTransferVars"
            },
            "CreateTransferVars": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "base_url + dest_path",
                        "arguments": {
                            "base_url.$": "$.cfde_ep_url",
                            "dest_path.$": "$.cfde_ep_path"
                        },
                        "result_path": "data_url"
                    }]
                },
                "ResultPath": "$.data_url",
                "WaitTime": 86400,
                "Next": "DerivaIngest"
            },
            "CreateNoTransferVars": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "data_url",
                        "arguments": {
                            "data_url.$": "$.data_url"
                        },
                        "result_path": "data_url"
                    }]
                },
                "ResultPath": "$.data_url",
                "WaitTime": 86400,
                "Next": "DerivaIngest"
            },
            "DerivaIngest": {
                "ActionScope": deriva_ap_scope,
                "ActionUrl": deriva_ap_url,
                #"ExceptionOnActionFailure": True,
                #TODO
                # Require new catalog, set ACL to author only
                "Parameters": {
                    "data_url.$": "$.data_url.details.data_url",
                    "operation": "ingest",
                    # "server": "demo.derivacloud.org",
                    # "catalog_id.$": ,
                    "catalog_acls.$": "$.vars.details.author_acls"
                },
                "ResultPath": "$.DerivaIngestResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CheckDerivaIngest"
            },
            "CheckDerivaIngest": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.DerivaIngestResult.details.error",
                    "BooleanEquals": False,
                    "Next": "CreateCurateText"
                }],
                "Default": "FailDerivaIngest"
            },
            "CreateCurateText": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": author_curate_text,
                        "arguments": {
                            "catalog_link.$": "$.DerivaIngestResult.details.deriva_link"
                        },
                        "result_path": "author"
                    }, {
                        "expression": curator_curate_text,
                        "arguments": {
                            "catalog_link.$": "$.DerivaIngestResult.details.deriva_link"
                        },
                        "result_path": "curator"
                    }]
                },
                "ResultPath": "$.CurateText",
                "WaitTime": 86400,
                "Next": "AuthorCurate"
            },
            "AuthorCurate": {
                "ActionScope": curate_subflow_scope,
                "ActionUrl": curate_subflow_url,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "curator_emails.$": "$._context.email",
                    #TODO
                    "curator_template": False,  # variables: $landing_page
                    #TODO
                    "curation_permissions": ["public"],
                    "curation_text.$": "$.CurateText.details.author",
                    "author_email": False,
                    "author_template": False,  # variables: $curation_task_id, $decision, $reason
                    "email_sender": sender_email,
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"]
                },
                "ResultPath": "$.AuthorCurateResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CheckAuthorCurate"
            },
            "CheckAuthorCurate": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.AuthorCurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "accepted",
                    "Next": "DerivaCuratorACL"
                }, {
                    "Variable": "$.AuthorCurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "rejected",
                    "Next": "FailCuration"
                }],
                "Default": "ErrorState"
            },
            "DerivaCuratorACL": {
                "Type": "Action",
                "ActionScope": deriva_ap_scope,
                "ActionUrl": deriva_ap_url,
                "ExceptionOnActionFailure": True,
                # Just set ACL to curator group
                "Parameters": {
                    "operation": "modify",
                    "catalog_id.$": "$.DerivaIngestResult.details.deriva_id",
                    "catalog_acls.$": "$.vars.details.curator_acls"
                },
                "ResultPath": "$.DerivaCuratorACLResult",
                "WaitTime": 86400,
                "Next": "CuratorCurate"
            },
            "CuratorCurate": {
                "ActionScope": curate_subflow_scope,
                "ActionUrl": curate_subflow_url,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "curator_emails": curator_emails,
                    #TODO
                    "curator_template": False,  # variables: $landing_page
                    # TODO
                    "curation_permissions": ["public"],
                    # "curation_permissions": [curator_group],
                    "curation_text.$": "$.CurateText.details.curator",
                    "author_email.$": "$._context.email",
                    #TODO
                    "author_template": False,  # variables: $curation_task_id, $decision, $reason
                    "email_sender": sender_email,
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"]
                },
                "ResultPath": "$.CuratorCurateResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CheckCuratorCurate"
            },
            "CheckCuratorCurate": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.CuratorCurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "accepted",
                    "Next": "DerivaCFDEACL"
                }, {
                    "Variable": "$.CuratorCurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "rejected",
                    "Next": "FailCuration"
                }],
                "Default": "ErrorState"
            },
            "DerivaCFDEACL": {
                "Type": "Action",
                "ActionScope": deriva_ap_scope,
                "ActionUrl": deriva_ap_url,
                "ExceptionOnActionFailure": True,
                # Just set ACL to CFDE group
                "Parameters": {
                    "operation": "modify",
                    "catalog_id.$": "$.DerivaIngestResult.details.deriva_id",
                    "catalog_acls.$": "$.vars.details.cfde_acls"
                },
                "ResultPath": "$.DerivaCFDEACLResult",
                "WaitTime": 86400,
                "Next": "CFDECurate"
            },
            "CFDECurate": {
                "ActionScope": curate_subflow_scope,
                "ActionUrl": curate_subflow_url,
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "curator_emails": cfde_emails,
                    #TODO
                    "curator_template": False,  # variables: $landing_page
                    # TODO
                    "curation_permissions": ["public"],
                    # "curation_permissions": [cfde_group],
                    "curation_text.$": "$.CurateText.details.curator",
                    "author_email.$": "$._context.email",
                    #TODO
                    "author_template": False,  # variables: $curation_task_id, $decision, $reason
                    "email_sender": sender_email,
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"]
                },
                "ResultPath": "$.CFDECurateResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CheckCFDECurate"
            },
            "CheckCFDECurate": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.CFDECurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "accepted",
                    "Next": "DerivaFinalACL"
                }, {
                    "Variable": "$.CFDECurateResult.details.output.CurationResult.details.name",
                    "StringEquals": "rejected",
                    "Next": "FailCuration"
                }],
                "Default": "ErrorState"
            },
            "DerivaFinalACL": {
                "Type": "Action",
                "ActionScope": deriva_ap_scope,
                "ActionUrl": deriva_ap_url,
                "ExceptionOnActionFailure": True,
                # Just set ACL to user's preference
                "Parameters": {
                    "operation": "modify",
                    "catalog_id.$": "$.DerivaIngestResult.details.deriva_id",
                    "catalog_acls.$": "$.vars.details.final_acls"
                },
                "ResultPath": "$.DerivaFinalACLResult",
                "WaitTime": 86400,
                "Next": "EmailSuccess"
            },
            "EmailSuccess": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template": success_email_template,
                    "body_variables": {
                        "action_id.$": "$._context.action_id",
                        "catalog_link.$": "$.DerivaFinalACLResult.details.deriva_link"
                    },
                    "destination.$": "$._context.email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Succeeded"
                },
                "ResultPath": "$.EmailSuccessResult",
                "WaitTime": 86400,
                "Next": "FlowSuccess"
            },
            "FlowSuccess": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "catalog_link",
                        "arguments": {
                            "catalog_link.$": "$.DerivaFinalACLResult.details.deriva_link"
                        },
                        "result_path": "deriva_link"
                    }, {
                        "expression": "catalog_id",
                        "arguments": {
                            "catalog_id.$": "$.DerivaFinalACLResult.details.deriva_id"
                        },
                        "result_path": "deriva_id"
                    }, {
                        "expression": ("'Submission Flow succeeded. Your catalog ID is ' + str(catalog_id) + "
                                       "', and your submission can be viewed at this link: ' + catalog_link"),
                        "arguments": {
                            "catalog_link.$": "$.DerivaFinalACLResult.details.deriva_link",
                            "catalog_id.$": "$.DerivaFinalACLResult.details.deriva_id"
                        },
                        "result_path": "message"
                    }]
                },
                "ResultPath": "$.SuccessState",
                "WaitTime": 86400,
                "Next": "FinishFlow"
            },
            "FailDerivaIngest": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": failure_text,
                        "arguments": {
                            "action_id.$": "$._context.action_id",
                            "error.$": "$.DerivaIngestResult.details.error"
                        },
                        "result_path": "error"
                    }]
                },
                "ResultPath": "$.FailureState",
                "WaitTime": 86400,
                "Next": "FailDerivaIngest2"
            },
            "FailDerivaIngest2": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.$": "$.FailureState.details.error",
                    "destination.$": "$._context.email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Failed to Ingest"
                },
                "ResultPath": "$.FailDerivaIngestResult",
                "WaitTime": 86400,
                "Next": "FinishFlow"
            },
            "FailCuration": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "text",
                        "arguments": {
                            "text": ("Your submission was rejected by a curator and did not complete the "
                                     "ingestion process.")
                        },
                        "result_path": "error"
                    }]
                },
                "ResultPath": "$.FailureState",
                "WaitTime": 86400,
                "Next": "FinishFlow"
            },
            "ErrorState": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.=": ("A CFDE Flow has errored. Please check the log for this Flow:\n"
                                        "Flow instance ID: `$._context.action_id`"),
                    "destination": admin_email,
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Failed to Ingest"
                },
                "ResultPath": "$.FailDerivaIngestResult",
                "WaitTime": 86400,
                "Next": "ErrorFlowLog"
            },
            "ErrorFlowLog": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "text",
                        "arguments": {
                            "text": ("A service error has occurred, and the CFDE team has been notified. "
                                     "You may be contacted with additional details.")
                        },
                        "result_path": "error"
                    }]
                },
                "ResultPath": "$.ErrorState",
                "WaitTime": 86400,
                "Next": "FinishFlow"
            },
            "FinishFlow": {
                "Type": "Pass",
                "End": True
            },
        }
    },
    "description": ("Run the full DERIVA ingest flow, with three stages of curation: Author, Curator, "
                    "and CFDE community."),
    "runnable_by": ["urn:globus:groups:id:a437abe3-c9a4-11e9-b441-0efb3ba9a670"],  # CFDE DERIVA Demo
    "synchronous": False,
    "title": "CFDE DERIVA Ingest with Curation",
    "types": ["Action", "Choice"],
    "visible_to": ["urn:globus:groups:id:a437abe3-c9a4-11e9-b441-0efb3ba9a670"]  # CFDE DERIVA Demo
}
input_schema = {
    # For Transfer from user (optional fields required if using Globus Transfer)
    "source_endpoint_id": "str or False",
    "source_path": "str, optional",
    "cfde_ep_id": "str, optional",  # "4674ef02-e1dc-4169-8303-f8ec1cbc3969",
    "cfde_ep_path": "str, optional",
    "cfde_ep_url": "str, optional",
    "is_directory": "str, optional",
    # For HTTP download (required if not using Globus Transfer)
    "data_url": "str, optional",
    # For DERIVA catalog
    "final_acls": "dict or False",
}
'''
# Curate Flow


# Deriva AP
"data_url"
"operation": ["restore", "ingest", "modify"]
"server"
"catalog_id"
"catalog_acls"

# Deriva Transfer Flow
"source_endpoint_id": "ce6d512a-b414-11e7-b0a7-22000a92523b",
"source_path": "/home/jgaff/Downloads/test_temp/CFDE-GTEx-v7.25194ab.C2M2.bdbag.tgz",
"fair_re_path": "/public/CFDE/ap_testing/test_flow1.bdbag.tgz",
"is_directory": False,
"restore": False
'''
pass

In [ ]:

full_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=full_curation_flow_def["definition"],
    title=full_curation_flow_def["title"],
    description=full_curation_flow_def["description"],
    visible_to=full_curation_flow_def["visible_to"],
    runnable_by=full_curation_flow_def["runnable_by"],
)
full_flow_id = full_flow_deploy_res["id"]
full_flow_scope = full_flow_deploy_res["globus_auth_scope"]

pass

In [ ]:
print(full_flow_id)
print(full_flow_scope)

In [ ]:
# Updating Flows causes errors - recommend redeploy instead
'''
full_flow_update_res = flows_client.update_flow(
    flow_id=full_flow_id,
    flow_definition=full_curation_flow_def["definition"],
    title=full_curation_flow_def["title"],
    description=full_curation_flow_def["description"],
    visible_to=full_curation_flow_def["visible_to"],
    runnable_by=full_curation_flow_def["runnable_by"],
)
'''

In [ ]:
#print(json.dumps(flows_client.get_flow(full_flow_id).data, indent=4, sort_keys=True))

In [ ]:
full_flow_input = {
    # For Transfer from user
    "source_endpoint_id": False,
    #"source_path": "str, optional",
    #"fair_re_path": "str, optional",
    #"is_directory": "str, optional",
    # For HTTP download
    # Bad data
    #"data_url": "https://317ec.36fe.dn.glob.us/public/CFDE/ingest_bags/hmp_noaux_170420.tgz",
    # Good data
    "data_url": "https://317ec.36fe.dn.glob.us/public/CFDE/metadata/datapackage_9ffed73c7ad3a52de17e467a8a5024af99d0ed84.zip",
    # For DERIVA catalog
    "final_acls": deriva_default_acls,
    # For curation
    "author_email": "jgaff@uchicago.edu"
}
"""
    "curator_email": "jgaff@uchicago.edu",
    "curator_template": False,
    "curation_text": "Custom curation text is here",
    "curation_task_id": "jgaff_flow_test_" + str(int(time.time())),
    "author_email": "jgaff@uchicago.edu",
    "author_template": "This is a test email for '$curation_task_id': Decision: $decision",
    "email_sender": "jgaff@uchicago.edu",
    "curation_permissions": ["public"],
    "aws_access_key_id": aws_id,
    "aws_secret_access_key": aws_secret,
    "aws_region": aws_region
}
"""
pass

In [ ]:
#full_flow_id = "bf06c589-15de-4fbc-af3e-2c74abae1311"
#full_flow_scope = "https://auth.globus.org/scopes/bf06c589-15de-4fbc-af3e-2c74abae1311/flow_bf06c589_15de_4fbc_af3e_2c74abae1311"

In [ ]:
full_flow_res = flows_client.run_flow(full_flow_id, full_flow_scope, full_flow_input)
full_flow_res.data

In [ ]:
status = flows_client.flow_action_status(full_flow_id, full_flow_scope, full_flow_res["action_id"]).data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flows_client.flow_action_log(full_flow_id, full_flow_scope, full_flow_res["action_id"], limit=100).data

In [ ]:
status = flows_client.flow_action_status(full_flow_id, full_flow_scope, "72a62c9f-dad6-4bc7-b3ef-1b37c5bb800c").data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flow_id = "7e94c079-fc4c-4b11-85dc-349301b7e607"
instance_id = "fbd15d5d-7ce7-409b-ab6d-935a01cac55f"
flow_scope = flows_client.get_flow(flow_id)["globus_auth_scope"]
flows_client.flow_action_log(flow_id,
                             flow_scope,
                             instance_id,
                             limit=100).data

In [ ]:
flows_client.get_flow(flow_id).data

In [ ]:
deriva_scope = "https://auth.globus.org/scopes/app.nih-cfde.org/deriva_all"
deriva_auth = mdf_toolbox.login(deriva_scope)[deriva_scope]
deriva_creds = {"bearer-token": deriva_auth.access_token}

In [ ]:
servername = "app-dev.nih-cfde.org"
server = DerivaServer("https", servername, deriva_creds)

In [ ]:
catalog_id = 36
catalog = server.connect_ermrest(catalog_id)

In [ ]:
delete_res = catalog.delete_ermrest_catalog(really=True)
print(delete_res)